<a href="https://colab.research.google.com/github/Manali-Patill/CODECRAFT_CS_02/blob/main/CODECRAFT_CS_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from PIL import Image
import random
import os

def generate_swap_indices(image_size):
    """
    Generate a list of swap indices for the image.
    This ensures that the swaps are reversible.
    """
    width, height = image_size
    total_pixels = width * height

    # Generate a list of pixel indices (0, 1, 2, ..., total_pixels-1)
    indices = list(range(total_pixels))

    # Shuffle the indices for encryption
    random.shuffle(indices)

    # Return the shuffled indices (used for encryption)
    return indices

def reverse_swap_indices(indices):
    """
    Reverse the swap order by finding the inverse of the swap indices.
    """
    reverse_indices = [0] * len(indices)
    for i, val in enumerate(indices):
        reverse_indices[val] = i
    return reverse_indices

def swap_pixels(image, swap_indices):
    """
    Apply pixel swapping based on the provided swap indices.
    This ensures the process is deterministic and can be reversed.
    """
    pixels = list(image.getdata())
    swapped_pixels = [pixels[i] for i in swap_indices]

    # Create a new image with the swapped pixels
    swapped_image = Image.new(image.mode, image.size)
    swapped_image.putdata(swapped_pixels)
    return swapped_image

def apply_value_transformation(image, key, operation="add"):
    """
    Apply a mathematical operation (add or subtract) to each pixel for encryption or decryption.
    """
    pixels = list(image.getdata())
    transformed_pixels = []

    for r, g, b in pixels:
        if operation == "add":
            transformed_pixels.append((
                (r + key) % 256,
                (g + key) % 256,
                (b + key) % 256
            ))
        elif operation == "subtract":
            transformed_pixels.append((
                (r - key) % 256,
                (g - key) % 256,
                (b - key) % 256
            ))

    # Create a new image from the transformed pixels
    transformed_image = Image.new(image.mode, image.size)
    transformed_image.putdata(transformed_pixels)
    return transformed_image

def save_image(image, file_path):
    """Helper function to save an image with error handling."""
    try:
        image.save(file_path)
        print(f"Image saved as {file_path}")
    except Exception as e:
        print(f"Error saving image: {e}")

def load_image(input_path):
    """Helper function to load an image with error handling."""
    if not os.path.exists(input_path):
        print(f"Error: The file '{input_path}' does not exist.")
        return None

    try:
        return Image.open(input_path)
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

def main():
    # Get user input for the image file path and encryption key
    input_path = input("Enter the path to the image you want to encrypt (e.g., 'input_image.jpg'): ")
    if not os.path.exists(input_path):
        print(f"Error: The file '{input_path}' does not exist.")
        return

    try:
        key = int(input("Enter an encryption/decryption key (e.g., 50): "))
    except ValueError:
        print("Invalid key. Please enter an integer.")
        return

    # Define default paths for encrypted and decrypted images
    encrypted_path = 'encrypted_image.jpg'
    decrypted_path = 'decrypted_image.jpg'

    # Load the image
    img = load_image(input_path)
    if img is None:
        return

    # Generate a deterministic swap sequence for encryption
    swap_indices = generate_swap_indices(img.size)

    # Encrypt the image by applying pixel swapping and value transformation
    encrypted_img = swap_pixels(img, swap_indices)
    encrypted_img = apply_value_transformation(encrypted_img, key, operation="add")
    save_image(encrypted_img, encrypted_path)

    # To decrypt, we need to reverse the pixel swaps.
    # First, reverse the swap indices to get back to the original order.
    reverse_swap = reverse_swap_indices(swap_indices)

    # Decrypt the image by reversing the operations
    decrypted_img = apply_value_transformation(encrypted_img, key, operation="subtract")
    decrypted_img = swap_pixels(decrypted_img, reverse_swap)  # Reverse the pixel swap
    save_image(decrypted_img, decrypted_path)

if __name__ == "__main__":
    main()


Enter the path to the image you want to encrypt (e.g., 'input_image.jpg'): input2.jpg
Enter an encryption/decryption key (e.g., 50): 12
Image saved as encrypted_image.jpg
Image saved as decrypted_image.jpg
